# MONGODB Setup Notebook

## Dockerfile generation

Docker file contains necessary commands needed to create any MongoDB instance

In [18]:
%%writefile Dockerfile

FROM mongo:6.0

# Copy the initialization script into the image
COPY mongo-init.sh /docker-entrypoint-initdb.d/mongo-init.sh

# Ensure the script has execute permissions
RUN chmod +x /docker-entrypoint-initdb.d/mongo-init.sh

Overwriting Dockerfile


## Creation of mongo-DB init script
Script used for initializing database

In [4]:
%%writefile mongo-init.sh

#!/bin/bash
set -e

# Ensure necessary environment variables are set
: "${MONGO_DATABASE_NAME:?Environment variable MONGO_DATABASE_NAME not set}"
: "${MONGO_DATABASE_USER:?Environment variable MONGO_DATABASE_USER not set}"
: "${MONGO_DATABASE_PASS:?Environment variable MONGO_DATABASE_PASS not set}"
: "${MONGO_INITDB_ROOT_USERNAME:?Environment variable MONGO_INITDB_ROOT_USERNAME not set}"
: "${MONGO_INITDB_ROOT_PASSWORD:?Environment variable MONGO_INITDB_ROOT_PASSWORD not set}"
: "${MONGO_COLLECTION_NAMES:?Environment variable MONGO_COLLECTION_NAMES not set}"

echo "Starting MongoDB initialization script..."

# Switch to admin database to create the app user
mongosh admin -u "$MONGO_INITDB_ROOT_USERNAME" -p "$MONGO_INITDB_ROOT_PASSWORD" --authenticationDatabase "admin" <<EOF
var user = db.getSiblingDB('$MONGO_DATABASE_NAME').getUser('$MONGO_DATABASE_USER');
if (!user) {
    // Create application database and user
    db.getSiblingDB('$MONGO_DATABASE_NAME').createUser({
      user: "$MONGO_DATABASE_USER",
      pwd: "$MONGO_DATABASE_PASS",
      roles: [
        { role: "readWrite", db: "$MONGO_DATABASE_NAME" }
      ]
    });
    print("Application user '$MONGO_DATABASE_USER' created successfully on '$MONGO_DATABASE_NAME'.");
} else {
    print("User '$MONGO_DATABASE_USER' already exists on '$MONGO_DATABASE_NAME'.");
}
EOF

# Initialize collections if they don't exist
mongosh "$MONGO_DATABASE_NAME" -u "$MONGO_DATABASE_USER" -p "$MONGO_DATABASE_PASS" --authenticationDatabase "$MONGO_DATABASE_NAME" <<EOF
var collections = $MONGO_COLLECTION_NAMES;
collections.forEach(function(collection) {
  if (db.getCollectionNames().indexOf(collection) === -1) {
  db.createCollection(collection);
  print("Collection '" + collection + "' created successfully in database '$MONGO_DATABASE_NAME'.");
  } else {
  print("Collection '" + collection + "' already exists in database '$MONGO_DATABASE_NAME'.");
  }
});
EOF

echo "MongoDB initialization complete."



Overwriting mongo-init.sh


## Local environment variables for MongoDB config
Defined in notebook for easier later generation

In [2]:
#Parking manager config data
mongo_service_name_managerDB= 'mongo_manager_db_service' 
mongo_db_name_managerDB = 'ParkMan_manager_db'
mongo_user_managerDB = 'user_Park_Manager'
mongo_pass_managerDB = 'mongo_pass'  
mongo_port_managerDB = '27016'
mongo_express_port_managerDB = '8081'
mongo_collections_managerDB = '["parking_lots", "parking_spaces", "parking_managers"]'


###Shared vals
mongo_admin_user = 'admin'
mongo_admin_pass = 'admin_pass'



mongo_express_user = 'mongo_express_user'
mongo_express_pass = 'mongo_express_pass'


###

#UserDB config data
mongo_service_name_userDB= 'mongo_user_db_service' 
mongo_db_name_userDB = 'ParkMan_user_db'
mongo_user_userDB = 'user_Person'
mongo_pass_userDB = 'mongo_pass'  
mongo_port_userDB = '27017'
mongo_express_port_userDB = '8082'
mongo_collections_userDB = '["users"]'

## Generate environment files for Docker definition

In [22]:
#append to docker-compose shared env neccessarry env variables so that the service could config properly
with open('../.env', 'a') as f:
    f.write(f"\n\nMONGO_PORT_MANAGERDB={mongo_port_managerDB}")
    f.write(f"\n\nMONGO_PORT_USERDB={mongo_port_userDB}")
    f.write(f"\nMONGO_EXPRESS_PORT_MANAGERDB={mongo_express_port_managerDB}")
    f.write(f"\nMONGO_EXPRESS_PORT_USERDB={mongo_express_port_userDB}")

Create .env file for mongoDB configuration (for managerDB and userDB)

In [6]:
#write env file for mongoDB ManagerDB config 
mongo_env = f'''MONGO_DATABASE_USER={mongo_user_managerDB}
MONGO_DATABASE_PASS={mongo_pass_managerDB}
MONGO_INITDB_ROOT_USERNAME={mongo_admin_user}
MONGO_INITDB_ROOT_PASSWORD={mongo_admin_pass}
MONGO_COLLECTION_NAMES={mongo_collections_managerDB}
'''
with open('.ManagerDB.env', 'w') as f:
    f.write(mongo_env)


#write env file for mongoDB UserDB config
mongo_env = f'''MONGO_DATABASE_USER={mongo_user_userDB}
MONGO_DATABASE_PASS={mongo_pass_userDB}
MONGO_INITDB_ROOT_USERNAME={mongo_admin_user}
MONGO_INITDB_ROOT_PASSWORD={mongo_admin_pass}
MONGO_COLLECTION_NAMES={mongo_collections_userDB}
'''
with open('.UserDB.env', 'w') as f:
    f.write(mongo_env)

Create .env file for Mongo Express config (for both instances)

In [7]:
#Create mongo express env for ManagerDB 
mongo_express_env = f'''ME_CONFIG_MONGODB_ADMINUSERNAME={mongo_admin_user}
ME_CONFIG_MONGODB_ADMINPASSWORD={mongo_admin_pass}
ME_CONFIG_MONGODB_URL=mongodb://{mongo_admin_user}:{mongo_admin_pass}@{mongo_service_name_managerDB}:27017/{mongo_db_name_managerDB}?authSource=admin
ME_CONFIG_BASICAUTH_USERNAME={mongo_express_user}
ME_CONFIG_BASICAUTH_PASSWORD={mongo_express_pass}
'''

with open('.env.express.ManagerDB', 'w') as f:
    f.write(mongo_express_env)

#Create mongo express env for UserDB
mongo_express_env = f'''ME_CONFIG_MONGODB_ADMINUSERNAME={mongo_admin_user}
ME_CONFIG_MONGODB_ADMINPASSWORD={mongo_admin_pass}
ME_CONFIG_MONGODB_URL=mongodb://{mongo_admin_user}:{mongo_admin_pass}@{mongo_service_name_userDB}:27017/{mongo_db_name_userDB}?authSource=admin
ME_CONFIG_BASICAUTH_USERNAME={mongo_express_user}
ME_CONFIG_BASICAUTH_PASSWORD={mongo_express_pass}
'''

with open('.env.express.UserDB', 'w') as f:
    f.write(mongo_express_env)


 




Add .env file for Flask app config

In [8]:
#Append to the ManagerApp env file the neccessary env variables
with open('../ManagerApp/.env', 'a') as f:
    f.write(f'\nMONGO_MANAGER_DATABASE_URI=mongodb://{mongo_user_managerDB}:{mongo_pass_managerDB}@{mongo_service_name_managerDB}:{mongo_port_managerDB}/{mongo_db_name_managerDB}')

#Append to the MobileApp env file the neccessary env variables
with open('../MobileApp/.env', 'a') as f:
    f.write(f'\nMONGO_USER_DATABASE_URI=mongodb://{mongo_user_userDB}:{mongo_pass_userDB}@{mongo_service_name_userDB}:{mongo_port_userDB}/{mongo_db_name_userDB}')

Add shared .env file needed for both apps and mongoDB config

In [26]:
#Create shared env file for ManagerApp and mongoDB config
shared_env_app = f'''
MONGO_DATABASE_NAME={mongo_db_name_managerDB}
'''  

with open('.env.shared.ManagerApp', 'w') as f:
    f.write(shared_env_app)

#Create shared env file for MobileApp and mongoDB config
shared_env_app = f'''
MONGO_DATABASE_NAME={mongo_db_name_userDB}
'''
with open('.env.shared.MobileApp', 'w') as f:
    f.write(shared_env_app)

Add shared .env file needed for both MongoDB and Mongo Express

In [27]:
#Create shared env file for Express and mongoDB config for managerDB
shared_env_express = f'''
MONGO_INITDB_DATABASE={mongo_db_name_managerDB}
'''

with open('.env.shared.express.ManagerDB', 'w') as f:
    f.write(shared_env_express)

#Create shared env file for Express and mongoDB config for userDB
shared_env_express = f'''
MONGO_INITDB_DATABASE={mongo_db_name_userDB}
'''

with open('.env.shared.express.UserDB', 'w') as f:
    f.write(shared_env_express)

# Docker compose generation
Cells bellow apply appropriate changes to docker-compose.yml

In [5]:

from ruamel.yaml import YAML

# Initialize YAML parser
yaml = YAML()
yaml.preserve_quotes = True  # Preserves quotes in the YAML file
yaml.indent(mapping = 2, sequence = 2, offset = 2)

#Setup file edit path 
docker_compose_path = '../docker-compose.yml'  

# Read the docker-compose.yml file
with open(docker_compose_path, 'r') as file:
    docker_compose = yaml.load(file)


#Define mongoDB service for ManagerDB
mongo_service_managerDB = {
    'build' : './mongoDB',
    'env_file' : [
        './mongoDB/.env.shared.express.ManagerDB',
        './mongoDB/.env.shared.ManagerApp',
        './mongoDB/.ManagerDB.env'
    ],
    'ports' : [
        "${MONGO_PORT_MANAGERDB}:27017"
    ],
    'volumes' : [
        'mongo_data_managerDB:/data/db'
    ],
    'command' : ["mongod", "--bind_ip_all"],
    'networks' : [
        'app-network'
    ],
    'healthcheck' : {
      'test' : ["CMD", "mongosh", "--eval","db.getMongo().getDB(process.env.MONGO_DATABASE_NAME) ? quit(0) : quit(1)"],
      'interval' : '10s',
      'timeout' : '5s',
      'retries' : 5
    }
}

#Define mongoDB service for UserDB
mongo_service_userDB = {
    'build' : './mongoDB',
    'env_file' : [
        './mongoDB/.env.shared.express.UserDB',
        './mongoDB/.env.shared.MobileApp',
        './mongoDB/.UserDB.env'
    ],
    'ports' : [
        "${MONGO_PORT_USERDB}:27017"
    ],
    'volumes' : [
        'mongo_data_userDB:/data/db' #TODO: check if this causes collisions/problems
    ],
    'command' : ["mongod", "--bind_ip_all"],
    'networks' : [
        'app-network'
    ],
    'healthcheck' : {
      'test' : ["CMD", "mongosh", "--eval","db.getMongo().getDB(process.env.MONGO_DATABASE_NAME) ? quit(0) : quit(1)"],
      'interval' : '10s',
      'timeout' : '5s',
      'retries' : 5
    }
}


#TODO: add MobileAPP requirements here
docker_compose['services']['manager_app']['depends_on'].update({ 

    mongo_service_name_managerDB: { #TODO: yaml parser has problems with parsing this line: it adds "-" to first item but not to second - verify why is that
        'condition': 'service_healthy'
        },
    mongo_service_name_userDB: {
        'condition': 'service_healthy'
        },
    
    }),
docker_compose['services']['manager_app']['env_file'] = [
    './ManagerApp/.env', 
    './mongoDB/.env.shared.ManagerApp'
]



# Define the Mongo-Express service configuration for ManagerDB and UserDB
mongo_express_service_managerDB = {
    'image' : 'mongo-express',
    'container_name' : 'mexpress', #TODO: maybe change container name in case of collision with mongo express for userDB
    'env_file' : [#TODO: modify so that there is only one .env.shared.express and .env.express
        './mongoDB/.env.express.ManagerDB',
        './mongoDB/.env.shared.express.ManagerDB'
    ],
    'restart' : 'unless-stopped',
    'ports': [
        "${MONGO_EXPRESS_PORT_MANAGERDB}:8081"
    ],
    'depends_on' : {
        mongo_service_name_managerDB: {
            'condition': 'service_healthy'
            },
        mongo_service_name_userDB: { 
            'condition': 'service_healthy'
            }        
    },
    'networks': [
        'app-network'
    ]
}

mongo_express_service_userDB = {
    'image' : 'mongo-express',
    'container_name' : 'mexpress_userDB', #TODO: maybe change container name in case of collision with mongo express for userDB
    'env_file' : [
        './mongoDB/.env.express.UserDB',
        './mongoDB/.env.shared.express.UserDB'
    ],
    'restart' : 'unless-stopped',
    'ports': [
        "${MONGO_EXPRESS_PORT_USERDB}:8081"
    ],
    'depends_on' : {
        mongo_service_name_managerDB: {
            'condition': 'service_healthy'
            },
        mongo_service_name_userDB: { 
            'condition': 'service_healthy'
            }        
    },
    'networks': [
        'app-network'
    ]
}




# Add the MongoDB service to the services section
docker_compose['services'][mongo_service_name_managerDB] = mongo_service_managerDB
docker_compose['services'][mongo_service_name_userDB] = mongo_service_userDB

# Add the Mongo-Express service to the services section
docker_compose['services']['mongo-express'] = mongo_express_service_managerDB
docker_compose['services']['mongo-express_userDB'] = mongo_express_service_userDB

docker_compose['volumes']['mongo_data_managerDB'] = {}
docker_compose['volumes']['mongo_data_userDB'] = {}

# Write the updated configuration back to docker-compose.yml
with open(docker_compose_path, 'w') as file:
    yaml.dump(docker_compose, file)

print("\ndocker-compose.yml has been updated successfully.")



docker-compose.yml has been updated successfully.
